Finding associations between the different items that are bought together. 

Trying out pythons apyori package for utilizing apriori algorithm for the task.

Both of the used datasets are small, and the results may thus be somewhat all over the place. This is mostly just to check out the mechanics of the apyori.

In [2]:
!pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=559e0ed1a1c7854d4940a0498bcc7fce82c83fd1927bdfae8931fc6a6e664329
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


Data from Kaggle: 
 https://www.kaggle.com/sivaram1987/association-rule-learningapriori/

 First upload kaggle.json

In [6]:
from google.colab import files

files.upload();

Saving kaggle.json to kaggle (1).json


In [7]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [11]:
!kaggle datasets download sivaram1987/association-rule-learningapriori
!unzip -q association-rule-learningapriori.zip

  0% 0.00/46.9k [00:00<?, ?B/s]
100% 46.9k/46.9k [00:00<00:00, 17.6MB/s]


In [12]:
!ls

 association-rule-learningapriori.zip   kaggle.json
'kaggle (1).json'		        sample_data


In [17]:
!head -n 5 Market_Basket_Optimisation.csv

shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
burgers,meatballs,eggs
chutney
turkey,avocado
mineral water,milk,energy bar,whole wheat rice,green tea


In [19]:
import pandas as pd
import numpy as np

from apyori import apriori

In [24]:
df = pd.read_csv("Market_Basket_Optimisation.csv", header=None)
df.shape

(7501, 20)

In [25]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


One row is one basket. Columns are different items. Colname 0 is the first item, colname 1 the 2nd and so on.

In [90]:
baskets = []

for row_idx, row in df.iterrows():
  baskets.append([item for item in row if not pd.isna(item)])

baskets[:5]

[['shrimp',
  'almonds',
  'avocado',
  'vegetables mix',
  'green grapes',
  'whole weat flour',
  'yams',
  'cottage cheese',
  'energy drink',
  'tomato juice',
  'low fat yogurt',
  'green tea',
  'honey',
  'salad',
  'mineral water',
  'salmon',
  'antioxydant juice',
  'frozen smoothie',
  'spinach',
  'olive oil'],
 ['burgers', 'meatballs', 'eggs'],
 ['chutney'],
 ['turkey', 'avocado'],
 ['mineral water', 'milk', 'energy bar', 'whole wheat rice', 'green tea']]

In [91]:
# Apply apriori algorithm

association_rules = apriori(baskets, min_support=0.005, min_confidence=0.25, 
                            min_lift=3, min_length=2)

association_results = list(association_rules)
association_results

[RelationRecord(items=frozenset({'mushroom cream sauce', 'escalope'}), support=0.005732568990801226, ordered_statistics=[OrderedStatistic(items_base=frozenset({'mushroom cream sauce'}), items_add=frozenset({'escalope'}), confidence=0.3006993006993007, lift=3.790832696715049)]),
 RelationRecord(items=frozenset({'pasta', 'escalope'}), support=0.005865884548726837, ordered_statistics=[OrderedStatistic(items_base=frozenset({'pasta'}), items_add=frozenset({'escalope'}), confidence=0.3728813559322034, lift=4.700811850163794)]),
 RelationRecord(items=frozenset({'herb & pepper', 'ground beef'}), support=0.015997866951073192, ordered_statistics=[OrderedStatistic(items_base=frozenset({'herb & pepper'}), items_add=frozenset({'ground beef'}), confidence=0.3234501347708895, lift=3.2919938411349285)]),
 RelationRecord(items=frozenset({'tomato sauce', 'ground beef'}), support=0.005332622317024397, ordered_statistics=[OrderedStatistic(items_base=frozenset({'tomato sauce'}), items_add=frozenset({'groun

In [92]:
# Collect found associations into a DF, sort by lift

def unpack_association_rules(association_results: list) -> pd.DataFrame:
  associations_df = pd.DataFrame(columns=('Items','Antecedent','Consequent','Support','Confidence','Lift'))

  sup = []
  conf = []
  lift = []
  items = []
  antec = []
  conseq = []

  for relation_record in association_results:
    for ordered_stat in relation_record.ordered_statistics:
        sup.append(relation_record.support)
        items.append(relation_record.items)
        antec.append(ordered_stat.items_base)
        conseq.append(ordered_stat.items_add)
        conf.append(ordered_stat.confidence)
        lift.append(ordered_stat.lift)

  associations_df['Items'] = list(map(set, items))                                   
  associations_df['Antecedent'] = list(map(set, antec))
  associations_df['Consequent'] = list(map(set, conseq))
  associations_df['Support'] = sup
  associations_df['Confidence'] = conf
  associations_df['Lift']= lift

  associations_df = associations_df.sort_values(by="Lift", ascending=False)
  
  return associations_df



In [95]:
associations_df = unpack_association_rules(association_results)
associations_df

,Items,Antecedent,Consequent,Support,Confidence,Lift
1,"{pasta, escalope}",{pasta},{escalope},0.005866,0.372881,4.700812
5,"{pasta, shrimp}",{pasta},{shrimp},0.005066,0.322034,4.506672
4,"{whole wheat pasta, olive oil}",{whole wheat pasta},{olive oil},0.007999,0.271493,4.122410
12,"{spaghetti, herb & pepper, ground beef}","{herb & pepper, spaghetti}",{ground beef},0.006399,0.393443,4.004360
11,"{mineral water, herb & pepper, ground beef}","{mineral water, herb & pepper}",{ground beef},0.006666,0.390625,3.975683
3,"{tomato sauce, ground beef}",{tomato sauce},{ground beef},0.005333,0.377358,3.840659
0,"{mushroom cream sauce, escalope}",{mushroom cream sauce},{escalope},0.005733,0.300699,3.790833
9,"{frozen vegetables, tomatoes, spaghetti}","{tomatoes, spaghetti}",{frozen vegetables},0.006666,0.318471,3.341054
2,"{herb & pepper, ground beef}",{herb & pepper},{ground beef},0.015998,0.323450,3.291994
10,"{ground beef, spaghetti, grated cheese}","{spaghetti, grated cheese}",{ground beef},0.005333,0.322581,3.283144


These are the strongest associations (that fulfill those conditions set to apriori): 
 - Pasta -> escalope
 - Pasta -> shrimp
 - Whole wheat pasta -> olive oil
 - etc.

## Let's try on another dataset:

In [63]:
!kaggle datasets download heeraldedhia/groceries-dataset
!unzip -q groceries-dataset.zip

  0% 0.00/257k [00:00<?, ?B/s]
100% 257k/257k [00:00<00:00, 17.1MB/s]


In [68]:
groceries_df = pd.read_csv("Groceries_dataset.csv")
groceries_df.shape

(38765, 3)

In [69]:
groceries_df

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk
...,...,...,...
38760,4471,08-10-2014,sliced cheese
38761,2022,23-02-2014,candy
38762,1097,16-04-2014,cake bar
38763,1510,03-12-2014,fruit/vegetable juice


In [70]:
groceries_df.columns = ["member_id", "date", "item"]
groceries_df

,member_id,date,item
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk
...,...,...,...
38760,4471,08-10-2014,sliced cheese
38761,2022,23-02-2014,candy
38762,1097,16-04-2014,cake bar
38763,1510,03-12-2014,fruit/vegetable juice


In [71]:
groceries_df.isna().sum()

member_id    0
date         0
item         0
dtype: int64

In [72]:
groceries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   member_id  38765 non-null  int64 
 1   date       38765 non-null  object
 2   item       38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB


We could form the baskets by grouping by member and date (assuming same member, same date rows belong to the same order). 

For the sake of doing something a bit different, let's ...

In [83]:
groceries_df.groupby(["member_id", "date"])["item"].size()

member_id  date      
1000       15-03-2015    4
           24-06-2014    3
           24-07-2015    2
           25-11-2015    2
           27-05-2015    2
                        ..
4999       24-01-2015    6
           26-12-2015    2
5000       09-03-2014    2
           10-02-2015    3
           16-11-2014    2
Name: item, Length: 14963, dtype: int64

In [87]:
groceries_baskets = []

for (m_id, dt), data_chunk in groceries_df.groupby(["member_id", "date"]):
  groceries_baskets.append(data_chunk["item"].tolist())

len(groceries_baskets)

14963

In [88]:
groceries_baskets[:5]

[['sausage', 'whole milk', 'semi-finished bread', 'yogurt'],
 ['whole milk', 'pastry', 'salty snack'],
 ['canned beer', 'misc. beverages'],
 ['sausage', 'hygiene articles'],
 ['soda', 'pickled vegetables']]

In [102]:
association_rules_groc = apriori(groceries_baskets, min_support=0.0003, min_confidence=0.25, 
                            min_lift=3, min_length=2)

association_results_groc = list(association_rules_groc) 

associations_groc_df = unpack_association_rules(association_results_groc)
associations_groc_df

,Items,Antecedent,Consequent,Support,Confidence,Lift
9,"{sausage, curd, fruit/vegetable juice}","{curd, fruit/vegetable juice}",{sausage},0.000334,0.500000,8.285161
0,"{canned beer, brown bread, frozen vegetables}","{frozen vegetables, brown bread}",{canned beer},0.000334,0.357143,7.612434
1,"{canned beer, brown bread, frozen vegetables}","{canned beer, frozen vegetables}",{brown bread},0.000334,0.250000,6.644316
21,"{soda, whole milk, pastry, other vegetables}","{soda, whole milk, other vegetables}",{pastry},0.000334,0.294118,5.685895
3,"{canned beer, yogurt, butter milk}","{yogurt, butter milk}",{canned beer},0.000334,0.263158,5.609162
19,"{soda, whole milk, pastry, other vegetables}","{soda, pastry, other vegetables}",{whole milk},0.000334,0.833333,5.276837
10,"{sausage, margarine, curd}","{margarine, curd}",{sausage},0.000334,0.312500,5.178225
20,"{soda, whole milk, pastry, other vegetables}","{whole milk, pastry, other vegetables}",{soda},0.000334,0.500000,5.149002
2,"{canned beer, yogurt, butter milk}","{canned beer, butter milk}",{yogurt},0.000334,0.416667,4.851816
8,"{pastry, sausage, coffee}","{sausage, coffee}",{pastry},0.000334,0.250000,4.833010
